In [69]:
import torch
import torchvision.transforms as transforms

In [70]:
# dir(transforms)

In [71]:
# torchvision.datasets.DatasetFolder(root, loader, extensions=None, transform=None, target_transform=None, is_valid_file=None)

In [72]:
# # https://discuss.pytorch.org/t/how-to-load-images-with-images-names-using-pytorch/47408/5
# from PIL import Image
# from torchvision.transforms import ToTensor, ToPILImage
# import numpy as np
# import random

# import tarfile
# import io
# import os
# import pandas as pd

# from torch.utils.data import Dataset
# import torch


# class YourDataset(Dataset):
#     def __init__(self, txt_path='filelist.txt', img_dir='data', transform=None):
#         """
#         Initialize data set as a list of IDs corresponding to each item of data set

#         :param img_dir: path to image files as a uncompressed tar archive
#         :param txt_path: a text file containing names of all of images line by line
#         :param transform: apply some transforms like cropping, rotating, etc on input image
#         """

#         df = pd.read_csv(txt_path, sep=' ', index_col=0)
#         self.img_names = df.index.values
#         self.txt_path = txt_path
#         self.img_dir = img_dir
#         self.transform = transform
#         self.to_tensor = ToTensor()
#         self.to_pil = ToPILImage()
#         self.get_image_selector = True if img_dir.__contains__('tar') else False
#         self.tf = tarfile.open(self.img_dir) if self.get_image_selector else None

#     def get_image_from_tar(self, name):
#         """
#         Gets a image by a name gathered from file list csv file

#         :param name: name of targeted image
#         :return: a PIL image
#         """
#         image = self.tf.extractfile(name)
#         image = image.read()
#         image = Image.open(io.BytesIO(image))
#         return image

#     def get_image_from_folder(self, name):
#         """
#         gets a image by a name gathered from file list text file

#         :param name: name of targeted image
#         :return: a PIL image
#         """

#         image = Image.open(os.path.join(self.img_dir, name))
#         return image

#     def __len__(self):
#         """
#         Return the length of data set using list of IDs

#         :return: number of samples in data set
#         """
#         return len(self.img_names)

#     def __getitem__(self, index):
#         """
#         Generate one item of data set.

#         :param index: index of item in IDs list

#         :return: a sample of data as a dict
#         """

#         if index == (self.__len__() - 1) and self.get_image_selector:  # close tarfile opened in __init__
#             self.tf.close()

#         if self.get_image_selector:  # note: we prefer to extract then process!
#             X = self.get_image_from_tar(self.img_names[index])
#         else:
#             X = self.get_image_from_folder(self.img_names[index])

#         # Get you label here using available pandas functions
#         Y = #########

#         if self.transform is not None:
#             X = self.transform(X)
# #            Y = self.transform(Y) # if your label is image too - remove if it is number

#         sample = {'X': X,
#                   'Y': Y}

#         return sample

In [73]:
# https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return X, y

In [74]:
import pandas as pd

In [75]:
train_list_df = pd.read_csv('train_val_list.txt', header=None, names=['Image Index'])

In [76]:
test_list_df = pd.read_csv('test_list.txt', header=None, names=['Image Index'])

In [77]:
train_list_df.head()

,Image Index
0,00000001_000.png
1,00000001_001.png
2,00000001_002.png
3,00000002_000.png
4,00000004_000.png


In [78]:
train_list_df.describe()

,Image Index
count,86524
unique,86524
top,00008004_005.png
freq,1


In [79]:
test_list_df.head()

,Image Index
0,00000003_000.png
1,00000003_001.png
2,00000003_002.png
3,00000003_003.png
4,00000003_004.png


In [80]:
test_list_df.describe()

,Image Index
count,25596
unique,25596
top,00002176_005.png
freq,1


In [81]:
xray_df = pd.read_csv('BBox_List_2017.csv')
xray_df = xray_df.dropna(axis=1)

In [82]:
xray_df.head()

,Image Index,Finding Label,Bbox [x,y,w,h]
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695


In [83]:
xray_df.groupby('Finding Label').describe()

Bbox [x                                                  \
                count        mean         std         min         25%   
Finding Label                                                           
Atelectasis     180.0  411.826190  228.271218   36.842328  212.385185   
Cardiomegaly    146.0  348.452582   55.593453  169.041270  321.627119   
Effusion        153.0  396.521930  294.312851    5.417989  136.533333   
Infiltrate      123.0  394.465529  202.291066   87.771429  207.278730   
Mass             85.0  434.307625  228.085662   70.433862  239.502222   
Nodule           79.0  502.405358  257.190738   68.266667  248.685714   
Pneumonia       120.0  378.674568  203.152877   40.093122  201.853968   
Pneumothorax     98.0  369.267215  224.407231   21.671958  170.646349   

                                                       y              ...  \
                      50%         75%         max  count        mean  ...   
Finding Label                                                         ...   
Atelectasis    309.552938  630.508475  892.203390  180.0  512.545597  ...   
Cardiomegaly   347.747969  383.220734  552.677966  146.0  392.336235  ...   
Effusion       199.680000  716.231111  863.004444  153.0  415.755706  ...   
Infiltrate     312.076190  591.644444  698.920635  123.0  389.101392  ...   
Mass           351.085714  649.075132  850.488889   85.0  366.680189  ...   
Nodule         614.400000  742.806349  905.887831   79.0  458.691099  ...   
Pneumonia      281.071746  589.876825  703.715556  120.0  416.731523  ...   
Pneumothorax   271.102646  580.774603  787.775661   98.0  209.356838  ...   

                        w                 h]                          \
                      75%         max  count        mean         std   
Finding Label                                                          
Atelectasis    245.358587  872.135593  180.0  139.110324   71.635608   
Cardiomegaly   512.000000  662.078307  146.0  381.118611   56.767949   
Effusion       265.102222  901.120000  153.0  318.007295  195.019538   
Infiltrate     262.230688  752.071111  123.0  297.393439  132.710229   
Mass           170.124868  698.595556   85.0  189.838700  133.644132   
Nodule          87.229630  122.446561   79.0   70.433862   23.756287   
Pneumonia      264.573968  763.448889  120.0  304.486942  141.120675   
Pneumothorax   261.688889  428.021164   98.0  246.010444  171.276897   

                                                                           
                      min         25%         50%         75%         max  
Finding Label                                                              
Atelectasis     29.582222   83.437037  117.633504  185.762712  439.182222  
Cardiomegaly   258.169492  344.044337  382.170281  415.830688  543.966138  
Effusion        51.200000  158.151111  277.401058  460.529101  873.379894  
Infiltrate      55.751111  192.338624  265.481481  394.429630  632.604444  
Mass            51.200000  101.858201  140.867725  231.889947  674.702222  
Nodule          26.006349   56.347090   68.266667   79.644444  150.620106  
Pneumonia       93.189418  194.411005  275.288042  378.121481  821.475556  
Pneumothorax    21.617778  120.401270  205.883598  299.073016  812.373333  

[8 rows x 32 columns]

In [87]:
xray2_df = pd.read_csv('Data_entry_2017_v2020.csv')

In [88]:
xray2_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168


In [90]:
xray2_df.groupby('Finding Labels').describe()

Follow-up #             \
                                                         count       mean   
Finding Labels                                                              
Atelectasis                                             4215.0   7.486358   
Atelectasis|Cardiomegaly                                  88.0   6.772727   
Atelectasis|Cardiomegaly|Consolidation                    10.0   4.100000   
Atelectasis|Cardiomegaly|Consolidation|Edema               1.0   6.000000   
Atelectasis|Cardiomegaly|Consolidation|Edema|Ef...         2.0   6.000000   
...                                                        ...        ...   
Pneumothorax                                            2194.0  14.175934   
Pneumothorax|Atelectasis                                   1.0  16.000000   
Pneumothorax|Infiltration                                  2.0  38.500000   
Pneumothorax|Mass                                          1.0   3.000000   
Pneumothorax|Pneumonia                                     2.0   1.500000   

                                                                            \
                                                          std   min    25%   
Finding Labels                                                               
Atelectasis                                         12.656322   0.0   1.00   
Atelectasis|Cardiomegaly                            11.487852   0.0   0.00   
Atelectasis|Cardiomegaly|Consolidation               4.306326   0.0   0.25   
Atelectasis|Cardiomegaly|Consolidation|Edema              NaN   6.0   6.00   
Atelectasis|Cardiomegaly|Consolidation|Edema|Ef...   8.485281   0.0   3.00   
...                                                       ...   ...    ...   
Pneumothorax                                        18.222770   0.0   3.00   
Pneumothorax|Atelectasis                                  NaN  16.0  16.00   
Pneumothorax|Infiltration                           26.162951  20.0  29.25   
Pneumothorax|Mass                                         NaN   3.0   3.00   
Pneumothorax|Pneumonia                               0.707107   1.0   1.25   

                                                                        \
                                                     50%    75%    max   
Finding Labels                                                           
Atelectasis                                          3.0   9.00  164.0   
Atelectasis|Cardiomegaly                             2.5   7.25   64.0   
Atelectasis|Cardiomegaly|Consolidation               3.0   6.00   11.0   
Atelectasis|Cardiomegaly|Consolidation|Edema         6.0   6.00    6.0   
Atelectasis|Cardiomegaly|Consolidation|Edema|Ef...   6.0   9.00   12.0   
...                                                  ...    ...    ...   
Pneumothorax                                         8.0  17.00  157.0   
Pneumothorax|Atelectasis                            16.0  16.00   16.0   
Pneumothorax|Infiltration                           38.5  47.75   57.0   
Pneumothorax|Mass                                    3.0   3.00    3.0   
Pneumothorax|Pneumonia                               1.5   1.75    2.0   

                                                   Patient ID                \
                                                        count          mean   
Finding Labels                                                                
Atelectasis                                            4215.0  14045.445789   
Atelectasis|Cardiomegaly                                 88.0  13766.931818   
Atelectasis|Cardiomegaly|Consolidation                   10.0  18905.100000   
Atelectasis|Cardiomegaly|Consolidation|Edema              1.0  12364.000000   
Atelectasis|Cardiomegaly|Consolidation|Edema|Ef...        2.0  15087.500000   
...                                                       ...           ...   
Pneumothorax                                           2194.0  14947.295351   
Pneumothorax|Atelectasis                            

In [94]:
xray2_df['Finding Labels'].unique()

array(['Cardiomegaly', 'Cardiomegaly|Emphysema', 'Cardiomegaly|Effusion',
       'No Finding', 'Hernia', 'Hernia|Infiltration', 'Mass|Nodule',
       'Infiltration', 'Effusion|Infiltration', 'Nodule', 'Emphysema',
       'Effusion', 'Atelectasis', 'Effusion|Mass', 'Infiltration|Mass',
       'Infiltration|Mass|Pneumothorax', 'Mass',
       'Cardiomegaly|Infiltration|Mass|Nodule',
       'Cardiomegaly|Effusion|Emphysema|Mass',
       'Atelectasis|Cardiomegaly|Emphysema|Mass|Pneumothorax',
       'Emphysema|Mass', 'Emphysema|Mass|Pneumothorax', 'Pneumothorax',
       'Emphysema|Pneumothorax', 'Atelectasis|Pneumothorax',
       'Cardiomegaly|Emphysema|Pneumothorax', 'Mass|Pleural_Thickening',
       'Cardiomegaly|Mass|Pleural_Thickening', 'Pleural_Thickening',
       'Effusion|Emphysema|Infiltration|Pneumothorax',
       'Emphysema|Infiltration|Pleural_Thickening|Pneumothorax',
       'Effusion|Pneumonia|Pneumothorax',
       'Effusion|Infiltration|Pneumothorax',
       'Effusion|Infiltra

In [91]:
xray2_df[xray2_df['Image Index'].isin(train_list_df['Image Index'])]

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143000,0.143000
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143000,0.143000
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168000,0.168000
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171000,0.171000
12,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168000,0.168000
...,...,...,...,...,...,...,...,...,...,...,...
112100,00030789_000.png,Infiltration,0,30789,51,F,PA,2021,2021,0.194311,0.194311
112106,00030793_000.png,Mass|Nodule,0,30793,57,F,PA,2021,2021,0.194311,0.194311
112108,00030795_000.png,Pleural_Thickening,0,30795,52,F,PA,2021,2021,0.194311,0.194311
112114,00030801_000.png,No Finding,0,30801,38,M,PA,2500,2048,0.168000,0.168000


In [85]:
xray_df[xray_df['Image Index'].isin(test_list_df['Image Index'])]

,Image Index,Finding Label,Bbox [x,y,w,h]
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695
...,...,...,...,...,...,...
979,00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889
980,00025769_001.png,Atelectasis,701.838229,572.491858,103.537778,63.715556
981,00016837_002.png,Atelectasis,140.913785,658.962969,271.928889,94.435556
982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778
